In [2]:
from utils import find_nearest_neighbors
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import os
# read in data
path_to_data = os.path.abspath('data')
path_to_colonies = os.path.abspath('data/colonies')

# nearest cell test on colony 0 to 5


In [3]:
colony0_segmentation_path = '/home/farzaneh/Documents/Bread/bread/src/bread/tests/data/V2022_09_19_HTB2_mCh_MYO1-GFP_50_ms/FOV0_segmentation_T0_to_T146_trimmed.h5'
colony0_lineage_GT_path = '/home/farzaneh/Documents/Bread/bread/src/bread/tests/data/V2022_09_19_HTB2_mCh_MYO1-GFP_50_ms/FOV0_lineage_T0_to_T146.csv'
colony0_nearest_cell_df = find_nearest_neighbors(colony0_segmentation_path, None)



/home/farzaneh/Documents/Bread/bread/src/bread/data/_data.py:596: MultipleContoursWarning: OpenCV returned multiple contours, 2 found.
  warnings.warn(Contour.MultipleContoursWarning(len(contours_cv)))
/home/farzaneh/Documents/Bread/bread/src/bread/data/_data.py:596: MultipleContoursWarning: OpenCV returned multiple contours, 4 found.
  warnings.warn(Contour.MultipleContoursWarning(len(contours_cv)))


In [4]:
colony0_nearest_cell_df

,bud_id,time_id,nearest_cell
0,1,0,3
1,2,0,4
2,3,0,1
3,4,0,2
4,5,18,1
...,...,...,...
363,364,146,204
364,365,146,169
365,366,146,111
366,367,146,5


In [5]:
colony0_lineage_GT = pd.read_csv(colony0_lineage_GT_path)

merged = pd.merge(colony0_nearest_cell_df, colony0_lineage_GT, on='bud_id', how='left')
merged = merged.loc[merged['parent_id']>0]
merged['colony'] = 0


In [6]:
merged

,bud_id,time_id,nearest_cell,parent_id,time_index,colony
4,5,18,1,1,18,0
5,6,18,3,3,18,0
6,7,20,2,2,20,0
7,8,22,4,4,22,0
8,9,32,1,1,32,0
...,...,...,...,...,...,...
363,364,146,204,91,146,0
364,365,146,169,132,146,0
365,366,146,111,111,146,0
366,367,146,5,224,146,0


In [7]:
print('accuracy of nearest cell for colony 0: ',accuracy_score(merged['nearest_cell'], merged['parent_id']))

accuracy of nearest cell for colony 0:  0.5386740331491713


In [8]:
# for colonies:
# colony 1 to 5
# load the 5 colony ground truth
colonies_gt = pd.DataFrame()
for i in [1, 2, 3, 4, 5]:
    temp_colony = pd.read_csv(os.path.join(
        path_to_colonies, 'colony00{}_lineage.csv'.format(i)))
    temp_colony['colony'] = i
    colonies_gt = pd.concat([colonies_gt, temp_colony])
colonies_gt = colonies_gt.reset_index(drop=True)
colonies_gt.rename(columns={'# parent_id': 'parent_GT'}, inplace=True)


# extract features for each colony and save them to a csv file. This takes a while. But after this, we only need to load the csv files. every time.
colony_nearest_cell = [0, 0, 0, 0, 0, 0]
args = {"nn_threshold": 8.0, "flexible_nn_threshold": True,
        "num_frames_refractory": 0, "num_frames": 4, "bud_distance_max": 10}
for i in [1, 2, 3, 4, 5]:
    print("finding features for colony {}".format(i))
    path = os.path.join(
        path_to_colonies, 'colony00{}_segmentation.h5'.format(i))
    colony_nearest_cell[i-1] = find_nearest_neighbors(path, None)
    colony_nearest_cell[i-1]['colony'] = i

colonies_nearest_cell = colony_nearest_cell[0]
for i in [1, 2, 3, 4]:
    temp_colony = colony_nearest_cell[i]
    colonies_nearest_cell = pd.concat([colonies_nearest_cell, temp_colony])
colonies_nearest_cell = colonies_nearest_cell.reset_index(drop=True)
colonies_nearest_cell = pd.merge(colonies_nearest_cell, colonies_gt, on=['bud_id', 'colony'], how='left')
colonies_nearest_cell = colonies_nearest_cell.loc[colonies_nearest_cell['parent_GT']>0]
colonies_nearest_cell

finding features for colony 1
finding features for colony 2
finding features for colony 3
finding features for colony 4
finding features for colony 5


,bud_id,time_id,nearest_cell,colony,parent_GT,time_index
2,3,4,2,1,2,4
3,4,7,1,1,1,7
4,5,27,4,1,4,27
5,6,27,1,1,1,27
6,7,28,2,1,2,28
...,...,...,...,...,...,...
499,34,164,22,5,22,164
500,35,166,13,5,13,166
501,36,169,18,5,18,169
502,37,175,18,5,3,175


In [15]:
print('accuracy of nearest cell for colony 1-5: ',accuracy_score(colonies_nearest_cell['nearest_cell'], colonies_nearest_cell['parent_GT']))


accuracy of nearest cell for colony 1-5:  0.702258726899384


In [9]:

print('accuracy of nearest cell for colony 1: ',accuracy_score(colonies_nearest_cell.loc[colonies_nearest_cell['colony']==1]['nearest_cell'], colonies_nearest_cell.loc[colonies_nearest_cell['colony']==1]['parent_GT']))
print('accuracy of nearest cell for colony 2: ',accuracy_score(colonies_nearest_cell.loc[colonies_nearest_cell['colony']==2]['nearest_cell'], colonies_nearest_cell.loc[colonies_nearest_cell['colony']==2]['parent_GT']))
print('accuracy of nearest cell for colony 3: ',accuracy_score(colonies_nearest_cell.loc[colonies_nearest_cell['colony']==3]['nearest_cell'], colonies_nearest_cell.loc[colonies_nearest_cell['colony']==3]['parent_GT']))
print('accuracy of nearest cell for colony 4: ',accuracy_score(colonies_nearest_cell.loc[colonies_nearest_cell['colony']==4]['nearest_cell'], colonies_nearest_cell.loc[colonies_nearest_cell['colony']==4]['parent_GT']))
print('accuracy of nearest cell for colony 5: ',accuracy_score(colonies_nearest_cell.loc[colonies_nearest_cell['colony']==5]['nearest_cell'], colonies_nearest_cell.loc[colonies_nearest_cell['colony']==5]['parent_GT']))

accuracy of nearest cell for colony 1:  0.7625
accuracy of nearest cell for colony 2:  0.7432432432432432
accuracy of nearest cell for colony 3:  0.6170212765957447
accuracy of nearest cell for colony 4:  0.6589147286821705
accuracy of nearest cell for colony 5:  0.7777777777777778


In [16]:
colony0_nearest_cell = merged.rename(columns={'parent_id': 'parent_GT'})
all_colonies_nearest_cell = pd.concat([colony0_nearest_cell, colonies_nearest_cell])

print('accuracy of nearest cell for all colonies: ',accuracy_score(all_colonies_nearest_cell['nearest_cell'], all_colonies_nearest_cell['parent_GT']))

accuracy of nearest cell for all colonies:  0.6325088339222615


# nearest cell test on external data


In [5]:
external_subset_lineage_GT_path_edited = '/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset/lineage/lineage_budlum_edited_by_code.csv'
external_subset_segmentation_path = '/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset/segmentation2/stacked/mask_stack _edited.h5' 

external_subset_lineage_gt= pd.read_csv(external_subset_lineage_GT_path_edited)

# find nearest cell for the external subset
external_subset_nearest_cell_df = find_nearest_neighbors(external_subset_segmentation_path, None)


merged = pd.merge(external_subset_nearest_cell_df, external_subset_lineage_gt, on='bud_id', how='left')


In [8]:
merged = merged.loc[merged['parent_GT']>0]

print('accuracy of nearest cell for external data subset: ',accuracy_score(merged['nearest_cell'], merged['parent_GT']))

accuracy of nearest cell for external data subset:  0.6496815286624203
